In [8]:
import requests
import pandas as pd
import scipy.spatial.distance as ssd

In [9]:
def query(postcode, offset=0):
  URL = 'https://api.ordnancesurvey.co.uk/places/v1/addresses/postcode'
  params = {'postcode': postcode, 'key': 'dxp6QRGemG64Aj4HdswNnIAzjIyKwMt1', 'offset': offset}
  data = requests.get(URL, params=params).json()
  header = data['header']
  results = data['results']
  yield [d['DPA'] for d in results]
  if header['totalresults'] > offset + header['maxresults']:
    yield from query(postcode, offset=offset + header['maxresults'])

In [25]:
from google.colab import files
uploaded = files.upload()

Saving PTALs.csv to PTALs.csv


In [30]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

ModuleNotFoundError: ignored

In [71]:
#from io import BytesIO
#ptal = pd.read_csv(BytesIO(uploaded['PTALs.csv']))
#points = ptal[['X', 'Y']].values

directory = input()
datapath= 'PTAL_Grid_2018_2021_2031.csv'
ptal_data = pd.read_csv(directory + datapath)
ptal_data.head()

ptal_data = ptal_data.rename(columns= {'X': 'X_COORDINATE', \
                                       'Y': 'Y_COORDINATE', \
                                       'PTALBY': 'PTAL_2018'})
points = ptal_data[['X_COORDINATE', 'Y_COORDINATE']]
print(points.iloc[:10, :])

/Users/vincentmargerin/CityDataHack/data/
   X_COORDINATE  Y_COORDINATE
0        527245        173952
1        527345        173952
2        527445        173952
3        527545        173952
4        527645        173952
5        527745        173952
6        527845        173952
7        527945        173952
8        528045        173952
9        528145        173952


In [72]:
def get_data(postcode):
  df = pd.concat(pd.DataFrame(res) for res in query(postcode))
  df['PTAL_2018'] = df.apply(lambda row: ptal_data.PTAL_2018[ssd.cdist([[row['X_COORDINATE'], row['Y_COORDINATE']]], points).argmin()], axis=1)
  return df

In [73]:
postcode = 'N5'
df = get_data(postcode)
df['Region'] = postcode

In [74]:
df= df.replace(to_replace= {'PTAL_2018': {'6a': '6', '6b': '7'}})
df['PTAL_2018'].value_counts()
df['PTAL_2018']= pd.to_numeric(df['PTAL_2018'])
df.describe()

,LOCAL_CUSTODIAN_CODE,MATCH,X_COORDINATE,Y_COORDINATE,PTAL_2018
count,10023.000000,10023.0,10023.000000,10023.000000,10023.000000
mean,5576.343909,1.0,531925.823106,185673.366936,4.701387
std,141.820190,0.0,380.010377,403.899651,1.293076
min,5360.000000,1.0,531183.000000,184748.000000,2.000000
25%,5570.000000,1.0,531600.000000,185316.000000,4.000000
50%,5570.000000,1.0,531918.000000,185701.000000,5.000000
75%,5570.000000,1.0,532225.000000,185996.000000,6.000000
max,7655.000000,1.0,532773.000000,186495.000000,7.000000


In [75]:
df.CLASSIFICATION_CODE_DESCRIPTION.value_counts()

Dwelling                                                           4748
Self Contained Flat (Includes Maisonette / Apartment)              3405
Property Shell                                                     1250
Terraced                                                            203
Shop / Showroom                                                     132
Dual Use                                                             47
Commercial                                                           40
Retail                                                               31
PO Box                                                               28
Office / Work Studio                                                 24
Detached                                                             17
Additional Mail / Packet Addressee                                   17
Semi-Detached                                                        11
Residential                                                     

In [76]:
datapath2= 'CO2_dataframe.csv'
co2_df = pd.read_csv(directory + datapath2, thousands=',')
co2_df = co2_df.rename(columns= {'Total': 'Total_CO2', \
                                 'Easting': 'X_COORDINATE', \
                                 'Northing': 'Y_COORDINATE'})
co2_df.head()
#co2_df['Total_CO2']= co2_df['Total_CO2'].apply(lambda x: x.replace('\t', ''))


,Year,LAEI 1km2 ID,GRID_ExactCut_ID,X_COORDINATE,Y_COORDINATE,Borough,Zone,Total_CO2
0,2008,5910,1,510500,203500,NonGLA,NonGLA,886.07
1,2008,5911,2,511500,203500,NonGLA,NonGLA,13999.04
2,2008,5912,3,512500,203500,NonGLA,NonGLA,7950.19
3,2008,5915,4,515500,203500,NonGLA,NonGLA,11873.98
4,2008,5916,5,516500,203500,NonGLA,NonGLA,5534.79


In [77]:
co2_points = co2_df[['X_COORDINATE', 'Y_COORDINATE']]
#print(co2_points.iloc[:10, :])
df['Total_CO2'] = df.apply(lambda row: co2_df.Total_CO2[ssd.cdist([[row['X_COORDINATE'], row['Y_COORDINATE']]], co2_points).argmin()], axis=1)
print(df.head())

                                             ADDRESS BLPU_STATE_CODE  \
0  FLAT 9, MATTHEWS COURT, HIGHBURY GRANGE, LONDO...             NaN   
1  FLAT 21, DRAKELEY COURT, AUBERT PARK, LONDON, ...             NaN   
2        54, PARK VIEW, COLLINS ROAD, LONDON, N5 2UD             NaN   
3         FLAT 11, 71A, DRAYTON PARK, LONDON, N5 1AN             NaN   
4       141, PARK VIEW, COLLINS ROAD, LONDON, N5 2UE             NaN   

  BLPU_STATE_CODE_DESCRIPTION BLPU_STATE_DATE   BUILDING_NAME BUILDING_NUMBER  \
0      Unknown/Not applicable             NaN  MATTHEWS COURT             NaN   
1      Unknown/Not applicable             NaN  DRAKELEY COURT             NaN   
2      Unknown/Not applicable             NaN             NaN              54   
3      Unknown/Not applicable             NaN             71A             NaN   
4      Unknown/Not applicable             NaN             NaN             141   

  CLASSIFICATION_CODE                    CLASSIFICATION_CODE_DESCRIPTION  \
0   

In [80]:
scores = df[['PTAL_2018', 'Total_CO2']]
ptal_co2 = pd.DataFrame.corr(scores)
print(ptal_co2)

           PTAL_2018  Total_CO2
PTAL_2018   1.000000   0.256944
Total_CO2   0.256944   1.000000


In [81]:
df.describe()

,LOCAL_CUSTODIAN_CODE,MATCH,X_COORDINATE,Y_COORDINATE,PTAL_2018,Total_CO2
count,10023.000000,10023.0,10023.000000,10023.000000,10023.000000,10023.000000
mean,5576.343909,1.0,531925.823106,185673.366936,4.701387,18647.472693
std,141.820190,0.0,380.010377,403.899651,1.293076,7683.895139
min,5360.000000,1.0,531183.000000,184748.000000,2.000000,4630.100000
25%,5570.000000,1.0,531600.000000,185316.000000,4.000000,19493.780000
50%,5570.000000,1.0,531918.000000,185701.000000,5.000000,19493.780000
75%,5570.000000,1.0,532225.000000,185996.000000,6.000000,24800.880000
max,7655.000000,1.0,532773.000000,186495.000000,7.000000,29136.180000


In [84]:
datapath3= 'ssx_openmapping_gb_v1.csv'
syntax_data = pd.read_csv(directory + datapath3)#, thousands=',')
#co2_df = co2_df.rename(columns= {'Total': 'Total_CO2', \
#                                 'Easting': 'X_COORDINATE', \
#                                 'Northing': 'Y_COORDINATE'})
#co2_df.head()
syntax_data.head()


,id,metres,meridian_id,meridian_gid,meridian_code,meridian_osodr,meridian_number,meridian_road_name,meridian_indicator,meridian_class,...,oa11cd,wz11cd,lsoa11cd,lsoa11nm,msoa11cd,msoa11nm,lad11cd,lad11nm,meridian_class_scale,wkt
0,5470,59.908263,761424,756472,3004,O16BHGHF0PPAW,NaN,STENTIFORD HILL ROAD,NaN,minor,...,E00076895,E33048014,E01015217,Torbay 008A,E02003161,Torbay 008,E06000027,Torbay,1,"LINESTRING(291820 63959,291805 64017)"
1,5471,99.126182,761424,756472,3004,O16BHGHF0PPAW,NaN,STENTIFORD HILL ROAD,NaN,minor,...,E00076895,E33048014,E01015217,Torbay 008A,E02003161,Torbay 008,E06000027,Torbay,1,"LINESTRING(291805 64017,291706 64022)"
2,5472,199.123078,761425,756473,3004,O16BGENY0X2AW,NaN,WHIDBORNE AVENUE,NaN,minor,...,E00077168,E33048019,E01015271,Torbay 010E,E02003163,Torbay 010,E06000027,Torbay,1,"LINESTRING(294362 63533,294207 63658)"
3,5473,241.615397,761425,756473,3004,O16BGENY0X2AW,NaN,WHIDBORNE AVENUE,NaN,minor,...,E00077168,E33048019,E01015271,Torbay 010E,E02003163,Torbay 010,E06000027,Torbay,1,"LINESTRING(294207 63658,293970 63611)"
4,5474,149.201206,761425,756473,3004,O16BGENY0X2AW,NaN,WHIDBORNE AVENUE,NaN,minor,...,E00077168,E33048019,E01015271,Torbay 010E,E02003163,Torbay 010,E06000027,Torbay,1,"LINESTRING(293970 63611,293851 63521)"


In [108]:
syntax_rank_data = syntax_data[['lsoa11cd', 'choice2kmrank', 'choice10kmrank', 'integration2kmrank', 'integration10kmrank']]
syntax_rank_data.head()


,lsoa11cd,choice2kmrank,choice10kmrank,integration2kmrank,integration10kmrank
0,E01015217,0.865533,0.646502,0.489954,0.446244
1,E01015217,0.877259,0.666571,0.586596,0.532800
2,E01015271,0.119891,0.315874,0.013624,0.061642
3,E01015271,0.090909,0.199432,0.004435,0.049503
4,E01015271,0.060241,0.056524,0.000000,0.044120


In [115]:
#datapath4 = 'd4278d47b8db3858f75e76803de4ce697.csv'
#geo_data = pd.read_csv(directory + datapath4)#, thousands=',')
#geo_data = geo_data.rename(columns= {'LSOA Code': 'lsoa11cd', \
#                                     'Easting': 'X_COORDINATE', \
#                                     'Northing': 'Y_COORDINATE'})
geo_data_thin = geo_data[['lsoa11cd', 'X_COORDINATE', 'Y_COORDINATE']]
geo_data_thin.head()

,lsoa11cd,X_COORDINATE,Y_COORDINATE
0,E01000675,540291,168873
1,E01000676,540262,169405
2,E01000675,540386,168710
3,E01000677,540197,169204
4,E01000675,540259,168855


In [118]:
choice_integration_data = syntax_rank_data.merge(geo_data_thin, on= ['lsoa11cd'], how= 'left').dropna()
choice_integration_data.head()

,lsoa11cd,choice2kmrank,choice10kmrank,integration2kmrank,integration10kmrank,X_COORDINATE,Y_COORDINATE
6589,E01032570,0.902098,0.770567,0.818182,0.12015,541886.0,158428.0
6590,E01032570,0.902098,0.770567,0.818182,0.12015,541886.0,158428.0
6591,E01032570,0.902098,0.770567,0.818182,0.12015,541886.0,158892.0
6592,E01032570,0.902098,0.770567,0.818182,0.12015,541984.0,158768.0
6593,E01032570,0.902098,0.770567,0.818182,0.12015,542184.0,158554.0


In [120]:
chint_points = choice_integration_data[['X_COORDINATE', 'Y_COORDINATE']]
#print(co2_points.iloc[:10, :])
df['integration2kmrank'] = df.apply(lambda row: choice_integration_data.integration2kmrank[ssd.cdist([[row['X_COORDINATE'], row['Y_COORDINATE']]], chint_points).argmin()], axis=1)
print(df.head())

KeyboardInterrupt: 

In [93]:
scores = df[['PTAL_2018', 'Total_CO2', 'choice2kmrank', 'choice10kmrank', 'integration2kmrank', 'integration10kmrank']]
all_n5_scores = pd.DataFrame.corr(scores)
print(all_n5_scores)

HIGH STREET          10699
STATION ROAD          7933
MAIN STREET           5044
CHURCH ROAD           4770
LONDON ROAD           4748
CHURCH LANE           4498
CHURCH STREET         3786
MILL LANE             3602
NEW ROAD              3208
GREEN LANE            2861
PARK ROAD             2764
MAIN ROAD             2739
THE STREET            2265
BACK LANE             2085
SCHOOL LANE           1798
VICTORIA ROAD         1767
PARK LANE             1696
MANOR ROAD            1646
LONG LANE             1387
SANDY LANE            1359
MOOR LANE             1345
MILL ROAD             1344
CHAPEL LANE           1317
WEST STREET           1306
KING STREET           1271
GRANGE ROAD           1224
NORTH ROAD            1213
THE AVENUE            1210
NORTH STREET          1203
THE CRESCENT          1192
                     ...  
DURLING STREET           1
MACQUARIE WAY            1
CRAIGNEIL STREET         1
BRAEBRIDGE PLACE         1
LAKIN STREET             1
ROWELL'S LANE            1
E

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (17,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,LSOA11cd,X_COORDINATE,Y_COORDINATE
0,E01000675,540291,168873
1,E01000676,540262,169405
2,E01000675,540386,168710
3,E01000677,540197,169204
4,E01000675,540259,168855
